In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [55]:
# Important Imports 
import glob

#Reading all files from given dataset 1 
path = r'/content/drive/MyDrive/Pattern Rec/training_validation' # use your path
all_files = glob.glob(path + "/class_[0,6]_*")

In [56]:
# Loading data files from dataset 1
print(f"total no of files found: {len(all_files)}")

main_data = pd.DataFrame(columns=range(0, 1025))
print(f"main dataframe created with shape: {main_data.shape}")

total no of files found: 384
main dataframe created with shape: (0, 1025)


In [6]:
# convert the value into list comma seperated value
def to_list(li):
  l = []
  for i in range(32):
    l.extend(list(li['data'][i]))

  return l

In [7]:
mydic = {}
def create_dict(myList, outputClass):
  for i in range(len(myList)+1):
    if i == 1024:
      mydic[i] = outputClass
    else:
      mydic[i] = myList[i]
    
  return mydic

In [8]:
for filenames in all_files:  
  # find the class name of the file
  class_name = str(filenames[(filenames.find('class_') + 6)])
  #print(f"Sample belongs to class: {class_name}")
  # pick one smaple file from class
  temp = pd.read_csv(filenames, names=["data"])
  to_append = to_list(temp)
  myDict = create_dict(to_append, class_name)
  main_data = main_data.append(myDict, ignore_index=True)

In [9]:
main_data.shape

(384, 1025)

In [10]:
# Seperating features and target class
X = main_data[[i for i in range(1024)]].astype(np.float32)
y = main_data[[1024]].to_numpy()
y = np.where( y == '6', '1', y)
y = y.ravel()

In [11]:
# Splitting data into test and train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=4) 

In [12]:
X_train = X_train.astype(int)
X_test = X_test.astype(int)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [42]:
# Defining accuracy function for our model
def accuracy(y_true, y_pred):
        accuracy = np.sum(y_true == y_pred) / len(y_true)
        return accuracy

In [48]:
# building own logistic regression
class logic():

  def __init__(self, lr=0.001, iter = 100):
   
    self.weights = None
    self.bias = None
    self.lr = lr
    self.iter = iter
    

  def fit(self, X, y):
      n_samples, n_features = X.shape
      self.bias = 0
      self.weights = np.zeros(n_features)
      
      # applying gradient descent to optimize the algorithm.
      for _ in range(self.iter):
          structure = np.dot(X, self.weights) + self.bias
          #apply sigmoid
          y_pred = self.sigmoid_activation(structure)
          dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
          db = (1 / n_samples) * np.sum(y_pred - y)
          # update parameters
          self.weights -= self.lr * dw
          self.bias -= self.lr * db
      print(f" final weight: {self.weights}")
      print(f"final bias:{self.bias}")
      print(f'Learning rate: {self.lr}') 
      print(f'epochs: {self.iter}')   
  
  def predict(self, X):
        y_predicted_cls = []  
        structure = np.dot(X, self.weights) + self.bias
        y_predicted = self.sigmoid_activation(structure)
        for i in y_predicted:
          if i > 0.5:
            y_predicted_cls.append(1)
          else:
            y_predicted_cls.append(0)  
        # y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        return np.array(y_predicted_cls)
  
 
  # The sigmoid activation function
  def sigmoid_activation(self, X_input):
          return 1. / (1. + np.exp(-X_input))

**REFERENCE:-** https://github.com/python-engineer/MLfromscratch/

In [50]:
# Creating object of model class
regre = logic(lr=0.01, iter=1000)

In [51]:
# Training model 
regre.fit(X_train, y_train)

 final weight: [ 0.00093043  0.00505931 -0.00590717 ...  0.          0.
  0.        ]
final bias:0.006488884521035741
Learning rate: 0.01
epochs: 1000


In [46]:
# Making prediction on test split of training data
prediction = regre.predict(X_test)

In [52]:
# Evaluating Model
print("LR classification accuracy:", accuracy(y_test, prediction) * 100)

LR classification accuracy: 100.0


In [26]:
# Loading the test file of Dataset 1 to Test our model
path = r'/content/drive/MyDrive/Pattern Rec/test' # use your path
new_test_file = glob.glob(path + "/class_[0,6]_*")
print(f"total no of files found: {len(new_test_file)}")

total no of files found: 174


In [29]:
test_data = pd.DataFrame(columns=range(0, 1025))
print(f"test dataframe created with shape: {test_data.shape}")

test dataframe created with shape: (0, 1025)


In [30]:
for filenames in new_test_file:
  # find the class name of the file
  class_name = str(filenames[(filenames.find('class_') + 6)])
  #print(f"Sample belongs to class: {class_name}")
  # pick one smaple file from class
  temp = pd.read_csv(filenames, names=["data"])
  to_append = to_list(temp)
  myDict = create_dict(to_append, class_name)
  test_data = test_data.append(myDict, ignore_index=True)

In [31]:
test_data.shape

(174, 1025)

In [32]:
# Seprating the test data into features and label
test_content = test_data[[i for i in range(0, 1024)]].to_numpy()
test_label = test_data[[1024]].to_numpy()
test_label = np.where( test_label == '6', '1', test_label)
test_label = test_label.ravel()

In [33]:
test_label=test_label.astype(int)
test_content=test_content.astype(int)

In [53]:
# prediction on Test data of dataset 1 
Test_pred = regre.predict(test_content)

In [54]:
# Evaluating model
test_acc = accuracy(test_label, Test_pred)
print(f'accuracy of the model on test data of dataset1 is: {test_acc * 100}')

accuracy of the model on test data of dataset1 is: 99.42528735632183
